<a href="https://colab.research.google.com/github/doooooodlesssss/imagecap/blob/main/imgcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ⚠️ Restart runtime first, then run this cleanly:
!pip install "numpy<1.27" opencv-python==4.6.0.66 yolov4

# Patch the YOLOv4 weights.py
file_path = "/usr/local/lib/python3.12/dist-packages/yolov4/tf/utils/weights.py"
with open(file_path, "r") as f:
    code = f.read()

code = code.replace(
    "conv_shape = (filters, conv2d.input_shape[-1], *conv2d.kernel_size)",
    "conv_shape = (filters, conv2d.input.shape[-1], *conv2d.kernel_size)"
)

with open(file_path, "w") as f:
    f.write(code)

print("✅ Patched YOLOv4 weights.py")

# Import YOLOv4 directly after patch
from yolov4.tf import YOLOv4


✅ Patched YOLOv4 weights.py
Call tf.config.experimental.set_memory_growth(GPU0, True)


In [3]:
repo_path = "/content/image_captioning"
%cd /content/drive/MyDrive/image_captioning/image_captioning/

kaggle_json = "/content/drive/MyDrive/image_captioning/kaggle.json"

dataset_dir = "/content/drive/MyDrive/image_captioning/datasets/coco2014"

feature_extraction_model = "xception"

model_dir = "trained_model_xception"

/content/drive/MyDrive/image_captioning/image_captioning


In [4]:
import tensorflow as tf
print("TensorFlow (preinstalled):", tf.__version__)
import os, importlib.util
import sys, pkgutil
print("Python:", sys.version)

def safe_pip(pkg, import_name=None):
    if import_name is None:
        import_name = pkg.split("==")[0]
    if importlib.util.find_spec(import_name) is None:
        !pip install {pkg}
    else:
        print(f"✔️ {pkg} already installed")

safe_pip("opencv-python-headless==4.6.0.66", "cv2")
safe_pip("matplotlib")
safe_pip("pillow", "PIL")
safe_pip("tqdm")
safe_pip("pycocotools")
safe_pip("lxml")
safe_pip("pandas")
safe_pip("seaborn")
safe_pip("yolov4")

if importlib.util.find_spec("tensorflow") is None:
    !pip install tensorflow
else:
    print("tensorflow already installed ✅")

if importlib.util.find_spec("kaggle") is None:
    !pip install kaggle
else:
    print("kaggle already installed ✅")



import numpy as np
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import datetime
import importlib


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re

import time
import json
from glob import glob

from tqdm import tqdm
from termcolor import colored
from builtins import len

from timeit import default_timer as timer
import threading

import cv2




TensorFlow (preinstalled): 2.19.0
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
✔️ opencv-python-headless==4.6.0.66 already installed
✔️ matplotlib already installed
✔️ pillow already installed
✔️ tqdm already installed
✔️ pycocotools already installed
✔️ lxml already installed
✔️ pandas already installed
✔️ seaborn already installed
✔️ yolov4 already installed
tensorflow already installed ✅
kaggle already installed ✅


In [20]:
# hyperparameters
WORD_DICT_SIZE = 15000
LIMIT_SIZE = True
EXAMPLE_NUMBER = 20000  # will only work if LIMIT_SIZE is True
MY_EMBEDDING_DIM = 256
UNIT_COUNT = 512
MY_OPTIMIZER = tf.keras.optimizers.Adam()
MY_LOSS_OBJECT = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
EPOCH_COUNT = 20
REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN = True
DATASET = "mscoco"  # "mscoco" or "flickr8k" or "flickr30k"
TEST_SET_PROPORTION = 1
feature_extraction_model = "xception"
split = 0  # 0 for training, 1 for testing

BATCH_SIZE = 32  # 64
BUFFER_SIZE = 1000  # 1000
embedding_dim = MY_EMBEDDING_DIM  # hadie
units = UNIT_COUNT  # hadie
top_k = WORD_DICT_SIZE
vocab_size = top_k + 1


In [21]:
# Python program for implementation of Quicksort Sort
def partition(arr, low, high):
	i = (low - 1)
	pivot = arr[high]
	for j in range(low, high):
		if arr[j][6] >= pivot[6]:
			i = i + 1
			arr[i], arr[j] = arr[j], arr[i]
	arr[i + 1], arr[high] = arr[high], arr[i + 1]
	return (i + 1)
def quickSort(arr, low, high):
	if len(arr) == 1:
		return arr
	if low < high:
		pi = partition(arr, low, high)
		quickSort(arr, low, pi - 1)
		quickSort(arr, pi + 1, high)

In [22]:
sys.path.append("/content/drive/MyDrive/image_captioning/image_captioning")

In [23]:
class BahdanauAttention(tf.keras.Model):

  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


class CNN_Encoder(tf.keras.Model):

    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x


class RNN_Decoder(tf.keras.Model):

  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the LSTM
    output, state, _ = self.lstm(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))


In [24]:
os.chdir(os.path.dirname(os.path.abspath("/content/drive/MyDrive/image_captioning/image_captioning")))
file = model_dir + "/max_length.txt"  # hadie
with open(file, 'r') as filetoread:  # hadie
    max_length = int(filetoread.readline())  # hadie

In [25]:
vocab_size = WORD_DICT_SIZE + 1
encoder = CNN_Encoder(MY_EMBEDDING_DIM)
decoder = RNN_Decoder(MY_EMBEDDING_DIM, UNIT_COUNT, vocab_size)
decoder.build(input_shape=(None, max_length)) #diya

decoder.load_weights(model_dir + "/my_model.weights.h5")

In [26]:
mod = importlib.import_module("feature_extraction_model_" + feature_extraction_model)  # hadie
image_model = mod.image_model  # hadie
load_image = mod.load_image  # hadie
attention_features_shape = mod.attention_features_shape + 1  # hadie

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [27]:
# loading the tokenizer
with open(model_dir + "/tokenizer.pickle", 'rb') as handle:
    tokenizer = pickle.load(handle)


def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

features_shape = 2048

In [28]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    yolo_features = image_path_to_yolo_bounding_boxes(image)  # hadie
    yolo_features = np.array(yolo_features.flatten())  # hadie
    yolo_features = np.pad(yolo_features, (0, features_shape - yolo_features.shape[0]), 'constant', constant_values=(0, 0)).astype(np.float32)  # hadie
    combined_features = np.vstack((img_tensor_val[0].numpy(), yolo_features)).astype(np.float32)  # hadie
    features = encoder(combined_features)  # hadie

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []


    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        attention_plot[i] = tf.reshape(attention_weights, (-1,)).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

        # result.append(tokenizer.index_word[predicted_id])

        # Safe mapping: replace out-of-vocab token IDs with <unk> #diya
        word = tokenizer.index_word.get(predicted_id, '<unk>')
        result.append(word)
        if word == '<end>':
            # return result, attention_plot
            break #diya



        # if tokenizer.index_word[predicted_id] == '<end>':
        #     return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result),:]
    return result, attention_plot




# result = []

# for i in range(max_caption_length):
#     predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

#     predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

#     # 🔒 Safe: replace unknown IDs with <unk>
#     word = tokenizer.index_word.get(predicted_id, '<unk>')
#     result.append(word)

#     # stop if <end> predicted
#     if word == '<end>':
#         break

#     # prepare next input
#     dec_input = tf.expand_dims([predicted_id], 0)


In [29]:
# # image = sys.argv[1]
# image = "/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014/COCO_val2014_000000000073.jpg"  # path to your image


# result, attention_plot = evaluate(image)
# file_name = ('caption_sample_' + str(datetime.datetime.now()) + ".png").replace(":", "_").replace(" ", "__")
# print ('Prediction Caption:`', ' '.join(result))
# # plt.savefig(file_name)
# print("The results have been written to " + file_name)
# # plot_attention(image, result, attention_plot)

In [30]:
yolo = YOLOv4()

# yolo = YOLOv4(tiny=True)

yolo.config.parse_names("/content/drive/MyDrive/image_captioning/image_captioning/coco.names")
yolo.config.parse_cfg("/content/drive/MyDrive/image_captioning/image_captioning/yolov4.cfg")
# yolo.input_size = (480,640)

yolo.make_model()
yolo.load_weights("/content/drive/MyDrive/image_captioning/weights/yolov4.weights", weights_type="yolo")

# yolo.inference(media_path="C:/Users/Hadie/Desktop/yolo/NYC_14th_Street_looking_west_12_2005.jpg")


# the output is sorted according to the area by confidence
def image_path_to_yolo_bounding_boxes(image_path):  # , coco_dict, word_index):
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    bboxes = yolo.predict(frame, prob_thresh=0.25)
    bboxes = bboxes.tolist()
    n = len(bboxes)
    # for each bounding box, append (area * confidence)
    for i in range(n):
        bboxes[i].append(bboxes[i][2] * bboxes[i][3] * bboxes[i][5])
        # obj_class_name = coco_dict[int(bboxes[i][4])].replace(" ", "")
        # if obj_class_name in word_index:
        #    bboxes[i][4] = word_index[coco_dict[int(bboxes[i][4])].replace(" ", "")]
        # else:
        #    bboxes[i][4] = word_index['<pad>']
    quickSort(bboxes, 0, n - 1)
    bboxes = np.array(bboxes)
    return bboxes


# raw feature extraction - not bounding boxes
def yolo_load_image(image_path):
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # height, width, _ = frame.shape
    frame = yolo.resize_image(frame)
    frame = frame / 255.0
    frame = frame[np.newaxis, ...].astype(np.float32)
    return frame




main code:

In [31]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError:
        pass


tf.get_logger().setLevel('ERROR')  # hadie


os.chdir("/content/drive/MyDrive/image_captioning/image_captioning")
sys.path.append(os.getcwd())  # add current folder to Python path

os.chdir(os.path.dirname(os.path.abspath("/content/drive/MyDrive/image_captioning/image_captioning")))  # hadie

from yolo import image_path_to_yolo_bounding_boxes  # hadie
start_date = datetime.datetime.now()  # hadie
my_start = timer()  # hadie

if not os.path.exists("trained_model_" + feature_extraction_model):  # create the dicrectory if it does not exists # hadie
    os.makedirs("trained_model_" + feature_extraction_model)  # hadie


In [32]:
# Annotation folder path
annotation_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/annotations/"

# Pick the right annotation file
if split == 0:  # training split
    annotation_file = os.path.join(annotation_folder, "captions_train2014.json")
    image_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/"

else:           # validation split
    annotation_file = os.path.join(annotation_folder, "captions_val2014.json")
    image_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014/"

PATH = image_folder


In [33]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)



all_captions = []
all_img_name_vector = []
all_ids = []  # hadie

image_id_index = {}  # hadie
for img in annotations['images']:  # hadie
    image_id_index[img['id']] = img['file_name']  # hadie

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    if DATASET == "mscoco":  # hadie
        full_coco_image_path = PATH + image_id_index[image_id]
        # print(full_coco_image_path, image_id)
    else:  # hadie
        full_coco_image_path = PATH + image_id + ".jpg"  # hadie
    all_ids.append(image_id)  # hadie
    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

train_ids, train_captions, img_name_vector = shuffle(all_ids,  # hadie
                                         all_captions,
                                         all_img_name_vector,
                                         random_state=1)

# Select the first 30000 captions from the shuffled set
num_examples = EXAMPLE_NUMBER  # hadie
if LIMIT_SIZE:  # hadie
    train_ids = train_ids[:num_examples]
    train_captions = train_captions[:num_examples]
    img_name_vector = img_name_vector[:num_examples]

print("training captions: ", len(train_captions), ", all captions: ", len(all_captions))  # hadie

training captions:  20000 , all captions:  414113


In [34]:
mod = importlib.import_module("feature_extraction_model_" + feature_extraction_model)  # hadie
image_model = mod.image_model  # hadie
load_image = mod.load_image  # hadie
attention_features_shape = mod.attention_features_shape + 1  # hadie
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [35]:
print("-----------------------------START OF EXECUTION-----------------------------")

# Get unique images
encode_train = sorted(set(img_name_vector))

# Only keep unprocessed images
encode_train = [
    x for x in encode_train
    if not os.path.exists(x + "_" + feature_extraction_model + ".npy")
]

# ✅ Filter out missing files
encode_train = [x for x in encode_train if os.path.exists(x)]

print(f"extracting features ({len(encode_train)}) valid file(s)")  # hadie

if len(encode_train) > 0:
    image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)

    # --- Safe loader that skips missing files ---
    def safe_load_image(path):
        try:
            img = tf.io.read_file(path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, (299, 299))
            img = tf.keras.applications.inception_v3.preprocess_input(img)
            return img, path
        except tf.errors.NotFoundError:
            # Skip missing or unreadable images
            print(f"⚠️ Skipping missing file: {path.numpy().decode('utf-8')}")
            return None

    def filter_none(data):
        return data is not None

    # Map safely
    image_dataset = (
        image_dataset
        .map(lambda path: tf.py_function(safe_load_image, [path], [tf.float32, tf.string]),
             num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .filter(lambda img, path: tf.reduce_all(tf.not_equal(tf.size(img), 0)))  # ensure valid tensors
        .batch(16)
    )

    for img, path in tqdm(image_dataset):
        batch_features = image_features_extract_model(img)
        batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

        for bf, p in zip(batch_features, path):
            path_of_feature = p.numpy().decode("utf-8")

            # --- YOLO + combined features ---
            yolo_features = image_path_to_yolo_bounding_boxes(path_of_feature)
            yolo_features = np.array(yolo_features).flatten()
            yolo_features = np.pad(yolo_features, (0, features_shape - len(yolo_features)),
                                   'constant', constant_values=(0, 0)).astype(np.float32)

            combined_features = np.vstack((bf.numpy(), yolo_features)).astype(np.float32)
            np.save(path_of_feature + "_" + feature_extraction_model, combined_features)

print("✅ finished extracting features")  # hadie


-----------------------------START OF EXECUTION-----------------------------
extracting features (0) valid file(s)
✅ finished extracting features


In [36]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


# Choose the top 5000 words from the vocabulary
top_k = WORD_DICT_SIZE  # hadie

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    print("using the cashed tokenizer")  # hadie
    # loading the tokenizer # hadie
    with open("trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'rb') as handle:  # hadie
        tokenizer = pickle.load(handle)  # hadie
else:  # hadie
    print("tokenizing and padding captions")  # hadie
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                      oov_token="<unk>",
                                                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(train_captions)
    train_seqs = tokenizer.texts_to_sequences(train_captions)  # 777 maybe this line needs removal
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    # saving the tokenizer to disk # hadie
    with open("trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'wb') as handle:  # hadie
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  # hadie

# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    file = "trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'r') as filetoread:  # hadie
        max_length = int(filetoread.readline())  # hadie
else:  # hadie
    # Calculates the max_length, which is used to store the attention weights
    max_length = calc_max_length(train_seqs)

    file = "trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'w') as filetowrite:  # hadie
        filetowrite.write(str(max_length))  # write the maximum length to disk # hadie

print("finished tokenizing and padding captions")  # hadie

"""## Split the data into training and testing"""

# # Create training and validation sets using an 80-20 split

# img_name_train, img_name_val, cap_train, cap_val = train_test_split(
#     img_name_vector, cap_vector, test_size=0.2, random_state=42
# )


# new spli w image_id_val
image_ids = np.arange(len(img_name_vector))  # assign IDs before splitting

image_id_train, image_id_val, img_name_train, img_name_val, cap_train, cap_val = train_test_split(
    image_ids, img_name_vector, cap_vector, test_size=0.2, random_state=42
)



tokenizing and padding captions


<>:18: SyntaxWarning: invalid escape sequence '\]'
<>:18: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-1048036035.py:18: SyntaxWarning: invalid escape sequence '\]'
  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')


finished tokenizing and padding captions


In [37]:
print("len(img_name_train) = ", len(img_name_train), ", len(cap_train) = ", len(cap_train), ", len(img_name_val) = ", len(img_name_val), ", len(cap_val) = ", len(cap_val))  # hadie

len(img_name_train) =  16000 , len(cap_train) =  16000 , len(img_name_val) =  4000 , len(cap_val) =  4000


In [38]:
num_steps = len(img_name_train) // BATCH_SIZE

def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8') + "_" + feature_extraction_model + '.npy')
  return img_tensor, cap


dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [39]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = MY_OPTIMIZER  # hadie
loss_object = MY_LOSS_OBJECT  # hadie


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [40]:
checkpoint_path = "./checkpoints/train"
# hadie
if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:
    try:
        for filename in os.listdir(checkpoint_path):
            print("deleting " + checkpoint_path + "/" + filename)
            os.unlink(checkpoint_path + "/" + filename)
    except Exception as e:
        # print('Failed to delete %s. Reason: %s' % (checkpoint_path + "/" + filename, e))
        print("Failed to delete checkpoint(s). Reason:", e)
    # remove the saved model too
    if os.path.exists("./trained_model_" + feature_extraction_model + "/my_model.index"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.index")
        os.unlink("./trained_model_" + feature_extraction_model + "/my_model.index")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/checkpoint"):
        print("deleting /trained_model_" + feature_extraction_model + "/checkpoint")
        os.unlink("./trained_model_" + feature_extraction_model + "/checkpoint")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
        os.unlink("./trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/learning_curve.png"):
        print("deleting trained_model_" + feature_extraction_model + "/learning_curve.png")
        os.unlink("./trained_model_" + feature_extraction_model + "/learning_curve.png")

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #diya

ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)


deleting ./checkpoints/train/ckpt-1.data-00000-of-00001
deleting ./checkpoints/train/ckpt-1.index
deleting ./checkpoints/train/checkpoint


In [41]:
print("🧹 Cleaning dataset — skipping missing .npy feature files...")

# Filter all valid feature files before creating dataset
valid_img_paths = []
missing_count = 0

for path in img_name_vector:
    feature_path = path + "_" + feature_extraction_model + ".npy"
    if os.path.exists(feature_path):
        valid_img_paths.append(path)
    else:
        missing_count += 1

print(f"✅ Found {len(valid_img_paths)} valid images.")
print(f"⚠️ Skipped {missing_count} missing feature files.\n")

# Build dataset from only valid files
img_name_train = valid_img_paths
cap_train = [cap for img, cap in zip(img_name_vector, cap_vector) if img in valid_img_paths]

# Now recreate dataset safely
def map_func(img_name, cap):
    # Decode bytes to string if needed
    img_path = img_name.numpy().decode('utf-8')
    feature_path = img_path + "_" + feature_extraction_model + ".npy"
    # guaranteed to exist from filtering above
    img_tensor = np.load(feature_path)
    return img_tensor, cap

def tf_map_func(img_name, cap):
    return tf.py_function(map_func, [img_name, cap], [tf.float32, tf.int32])

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.map(tf_map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(64)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print("✅ Dataset built successfully — all missing .npy files skipped.")


🧹 Cleaning dataset — skipping missing .npy feature files...
✅ Found 11987 valid images.
⚠️ Skipped 8013 missing feature files.

✅ Dataset built successfully — all missing .npy files skipped.


In [42]:
# print(f"📸 Total test images found: {len(img_name_test)}")

In [43]:
loss_plot = []

@tf.function
def train_step(img_tensor, target):
    loss = 0
    hidden = decoder.reset_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        for i in range(1, target.shape[1]):
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = loss / int(target.shape[1])
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss


EPOCHS = 20  # hadie

# 🧠 Helper to safely load batches
def safe_batch_loader(dataset):
    """Yield batches, skipping missing files gracefully."""
    for batch, (img_tensor, target) in enumerate(dataset):
        try:
            yield batch, (img_tensor, target)
        except (FileNotFoundError, tf.errors.NotFoundError) as e:
            print(f"⚠️ Skipping missing or corrupted file in batch {batch}: {e}")
            continue


if not os.path.exists("trained_model_" + feature_extraction_model + "/my_model.index"):  # hadie
    print("training..")  # hadie
    for epoch in range(start_epoch, EPOCHS):
        print(f"Starting epoch {epoch+1}/{EPOCHS}")
        start = time.time()
        total_loss = 0

        for batch, (img_tensor, target) in safe_batch_loader(dataset):
            try:
                batch_loss, t_loss = train_step(img_tensor, target)
                total_loss += t_loss
            except (FileNotFoundError, tf.errors.NotFoundError) as e:
                print(f"⚠️ Skipping batch {batch} due to missing file: {e}")
                continue

            if batch % 100 == 0:
                print(f"Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}")
                print('Epoch {} Batch {} Loss {:.4f}'.format(
                    epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))

        if len(dataset) > 0:
            avg_loss = total_loss / len(dataset)
        else:
            avg_loss = 0

        print(f"Epoch {epoch+1} Loss {avg_loss:.4f}")
        loss_plot.append(avg_loss)

        if epoch % 5 == 0:
            ckpt_manager.save()

        print(f"Epoch {epoch+1} Loss {avg_loss:.6f}")
        print(f"Time taken for 1 epoch {time.time() - start:.2f} sec\n")

    # Save model
    save_dir = f"trained_model_{feature_extraction_model}"

    base_dir = "/content/drive/MyDrive/image_captioning"

    save_dir = os.path.join(base_dir, f"trained_model_{feature_extraction_model}")

    os.makedirs(save_dir, exist_ok=True)
    decoder.save_weights(os.path.join(save_dir, "my_model.weights.h5"))

else:  # hadie
    print("A trained model has been found. Loading it from disk..")  # hadie
    decoder.load_weights("trained_model_" + feature_extraction_model + "/my_model.weights.h5")


training..
Starting epoch 1/20
Epoch 1 Batch 0 Loss 113.1457
Epoch 1 Batch 0 Loss 2.3091
Epoch 1 Batch 100 Loss 56.2297
Epoch 1 Batch 100 Loss 1.1475
Epoch 1 Loss 1.2239
Epoch 1 Loss 1.223855
Time taken for 1 epoch 3139.80 sec

Starting epoch 2/20
Epoch 2 Batch 0 Loss 51.4264
Epoch 2 Batch 0 Loss 1.0495
Epoch 2 Batch 100 Loss 45.1979
Epoch 2 Batch 100 Loss 0.9224
Epoch 2 Loss 0.9717
Epoch 2 Loss 0.971686
Time taken for 1 epoch 150.91 sec

Starting epoch 3/20
Epoch 3 Batch 0 Loss 42.7482
Epoch 3 Batch 0 Loss 0.8724
Epoch 3 Batch 100 Loss 42.1084
Epoch 3 Batch 100 Loss 0.8594
Epoch 3 Loss 0.8476
Epoch 3 Loss 0.847588
Time taken for 1 epoch 149.75 sec

Starting epoch 4/20
Epoch 4 Batch 0 Loss 41.9912
Epoch 4 Batch 0 Loss 0.8570
Epoch 4 Batch 100 Loss 38.5713
Epoch 4 Batch 100 Loss 0.7872
Epoch 4 Loss 0.7729
Epoch 4 Loss 0.772911
Time taken for 1 epoch 150.70 sec

Starting epoch 5/20
Epoch 5 Batch 0 Loss 32.9892
Epoch 5 Batch 0 Loss 0.6732
Epoch 5 Batch 100 Loss 35.6564
Epoch 5 Batch 100 L

In [44]:
encoder.save_weights(os.path.join(save_dir, "encoder.weights.h5"))
decoder.save_weights(os.path.join(save_dir, "decoder.weights.h5"))

In [45]:
with open(os.path.join(save_dir, "loss_plot.json"), "w") as f:
    json.dump([float(l) for l in loss_plot], f)

In [46]:
with open(os.path.join(save_dir, "tokenizer.pkl"), "wb") as f:
    pickle.dump(tokenizer, f)

In [47]:
#load loss and tokeniser
save_dir = f"trained_model_{feature_extraction_model}"

with open(os.path.join(save_dir, "loss_plot.json")) as f:
    loss_plot = json.load(f)

with open(os.path.join(save_dir, "tokenizer.pkl"), "rb") as f:
    tokenizer = pickle.load(f)


In [48]:
# def evaluate(image):
#     attention_plot = np.zeros((max_length, attention_features_shape))

#     hidden = decoder.reset_state(batch_size=1)

#     temp_input = tf.expand_dims(load_image(image)[0], 0)
#     img_tensor_val = image_features_extract_model(temp_input)
#     img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

#     # yolo_features = image_path_to_yolo_bounding_boxes(image.decode("utf-8"))  # hadie

#     yolo_features = image_path_to_yolo_bounding_boxes(image) #diya


#     yolo_features = np.array(yolo_features.flatten())  # hadie
#     yolo_features = np.pad(yolo_features, (0, features_shape - yolo_features.shape[0]), 'constant', constant_values=(0, 0)).astype(np.float32)  # hadie
#     combined_features = np.vstack((img_tensor_val[0].numpy(), yolo_features)).astype(np.float32)  # hadie
#     features = encoder(combined_features)  # hadie

#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

#     result = []

#     for i in range(max_length):
#         predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
#         attention_plot[i] = tf.reshape(attention_weights, (-1,)).numpy()

#         predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
#         result.append(tokenizer.index_word[predicted_id])

          # # Safe mapping: replace out-of-vocab token IDs with <unk> #diyaaa
          # word = tokenizer.index_word.get(predicted_id, '<unk>')
          # result.append(word)

          # if word == '<end>':
          #     return result, attention_plot


#         if tokenizer.index_word[predicted_id] == '<end>':
#             return result, attention_plot

#         dec_input = tf.expand_dims([predicted_id], 0)

#     attention_plot = attention_plot[:len(result),:]
#     return result, attention_plot



def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [49]:
print("Total img_name_val:", len(img_name_val))
print("Example path:", img_name_val[0])

Total img_name_val: 4000
Example path: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000124931.jpg


In [50]:
def validate_image(id, image_name_for_validation, original_caption):
  try:
    print("Evaluating:", image_name_for_validation.decode("utf-8"), "From thread", threading.current_thread().ident)

    result, attention_plot = evaluate(image_name_for_validation)  # generate the hypothesis
    result = ' '.join(result).replace("<end>", "").strip()  # remove unnecessary characters

    dict = {}
    dict["image_id"] = int(id)  # .decode("utf-8")
    dict["caption"] = result;
    dict["original_caption"] = original_caption.decode("utf-8")
    dict["file_name"] = image_name_for_validation.decode("utf-8")
    return json.dumps(dict)
  except Exception as e:
        print("⚠️ Skipping image:", image_name_for_validation.decode("utf-8"), "Error:", e)
        return json.dumps({})  # return an empty dict so the pipeline doesn't break

caption_strings_val = list(map(lambda item: (' '.join([tokenizer.index_word[i] for i in item if i not in [0]])).replace("<end>", "").replace("<start>", "").strip(), cap_val))  # convert to a list of strings
validation_dataset = tf.data.Dataset.from_tensor_slices((image_id_val, img_name_val, caption_strings_val))  # create the dataset
validation_dataset = validation_dataset.take(300)  # only evaluate first 100 images
eval_start_date = datetime.datetime.now()  # hadie
list_of_dicts = validation_dataset.map(lambda item1, item2, item3: tf.numpy_function(validate_image, [item1, item2, item3], [tf.string]), num_parallel_calls=1)  # run in parallel
list_of_dicts = list(list_of_dicts.as_numpy_iterator())  # convert to a list
list_of_dicts = [item for sublist in list_of_dicts for item in sublist]  # flatten the list by removing nested tuples
list_of_dicts = list(map(lambda item: json.loads(item), list_of_dicts))  # rewrap the strings as dictionaries, then convert to a list

added_ids = []

# remove empty dicts (those from skipped images)
list_of_dicts = [d for d in list_of_dicts if "image_id" in d]

unique_list_of_dicts = []
for d in list_of_dicts:
    if d["image_id"] not in added_ids:
        added_ids.append(d["image_id"])
        unique_list_of_dicts.append(d)


unique_list_of_dicts = []
for dict in list_of_dicts:
    if not dict["image_id"] in added_ids:
        added_ids.append(dict["image_id"])
        unique_list_of_dicts.append(dict)

file = "trained_model_" + feature_extraction_model + "/results.json"  # hadie
with open(file, 'w') as filetowrite:  # hadie
    filetowrite.write(json.dumps(unique_list_of_dicts))  # hadie

print("The results have been written to trained_model_" + feature_extraction_model + "/results.json")  # hadie
print("Main thread:", threading.current_thread().ident)  # hadie

end_date = datetime.datetime.now()  # hadie

my_end = timer()  # hadie
hours, rem = divmod(my_end - my_start, 3600)  # hadie
minutes, seconds = divmod(rem, 60)  # hadie

print("Start time: " + str(start_date))  # hadie
print("Evaluation start time: " + str(eval_start_date))  # hadie
print("End time: " + str(end_date))  # hadie

print("Time elapsed (hours:minutes:seconds): {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))  # hadie

if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    plt.plot(loss_plot)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.savefig("trained_model_" + feature_extraction_model + "/learning_curve.png")  # hadie
    # plt.show() # hadie

print("The learning curve has been written to trained_model_" + feature_extraction_model + "/learning_curve.png")  # hadie
print("Feature extraction model: " + feature_extraction_model)
print("Dataset: " + DATASET)
print("Development set proportion: " + str(TEST_SET_PROPORTION))
print("with yolo bounding boxes")

Evaluating: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000124931.jpg From thread 139674021525056
⚠️ Skipping image: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000124931.jpg Error: Can't convert object to 'str' for 'filename'
Evaluating: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000389698.jpg From thread 139673728964160
⚠️ Skipping image: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000389698.jpg Error: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000389698.jpg; No such file or directory [Op:ReadFile]
Evaluating: /content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/COCO_train2014_000000002007.jpg From thread 139674021525056
⚠️ Skipping image: /content/drive/MyDrive/image_capt

KeyboardInterrupt: 

In [51]:
test_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014"  # or test2014 if you have it


img_name_test = sorted([
    os.path.join(test_folder, f)
    for f in os.listdir(test_folder)
    if f.lower().endswith(".jpg")
])

# ✅ 3. Filter only those with extracted .npy features
img_name_test = [
    path for path in img_name_test
    if os.path.exists(path + "_" + feature_extraction_model + ".npy")
]

In [52]:
import json, os
from tqdm import tqdm
import numpy as np
import tensorflow as tf

results = []

# Directory where results.json should be saved
results_path = os.path.join(save_dir, "results.json")

# ✅ Ensure you have a loaded model (encoder + decoder)
encoder.load_weights(os.path.join(save_dir, "encoder.weights.h5"))
decoder.load_weights(os.path.join(save_dir, "decoder.weights.h5"))

def evaluate_image(image_path):
    """Generate a caption for one image."""
    # Load precomputed features
    img_tensor = np.load(image_path + "_" + feature_extraction_model + ".npy")
    img_tensor = tf.convert_to_tensor(img_tensor)
    img_tensor = tf.expand_dims(img_tensor, 0)

    features = encoder(img_tensor)
    hidden = decoder.reset_state(batch_size=1)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(40):  # max caption length
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        word = tokenizer.index_word.get(predicted_id, '')
        if word == '<end>':
            break
        result.append(word)
        dec_input = tf.expand_dims([predicted_id], 0)

    return ' '.join(result)

# ✅ Evaluate a subset or all test images
test_images = img_name_test[:50]  # or full test set
for img_path in tqdm(test_images):
    if not os.path.exists(img_path + "_" + feature_extraction_model + ".npy"):
        continue
    predicted_caption = evaluate_image(img_path)
    results.append({
        "image_id": os.path.basename(img_path),
        "predicted_caption": predicted_caption
    })

# ✅ Save results to Drive
with open(results_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ Saved {len(results)} predictions to {results_path}")


100%|██████████| 50/50 [00:47<00:00,  1.05it/s]


✅ Saved 50 predictions to trained_model_xception/results.json


In [53]:
# Load COCO annotations
with open('/content/drive/MyDrive/image_captioning/datasets/coco2014/annotations/captions_val2014.json') as f:
    val_data = json.load(f)

results = []

# Evaluate on a small subset first (say 500 images for speed)
for ann in tqdm(val_data['annotations'][:500]):
    img_id = ann['image_id']
    img_path = f"/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014/COCO_val2014_{img_id:012d}.jpg"

    # Get predicted caption
    pred_caption, _ = evaluate(img_path)
    pred_caption = ' '.join(pred_caption).replace('<start>', '').replace('<end>', '').strip()

    results.append({
        "image_id": img_id,
        "caption": pred_caption
    })

# Save predictions in COCO format
with open("results.json", "w") as f:
    json.dump(results, f)


100%|██████████| 500/500 [05:30<00:00,  1.51it/s]


In [54]:
print(results[:5])

[{'image_id': 203564, 'caption': 'a man sitting on a bike'}, {'image_id': 179765, 'caption': 'a tree parked in a parked in a city'}, {'image_id': 322141, 'caption': 'a bunch of an image of a bathroom toilet and a window as the wall sink in a mirror'}, {'image_id': 16977, 'caption': 'a patio view of a traffic traffic light no parking meter'}, {'image_id': 106140, 'caption': 'a icelandair airliner is getting ready to take off'}]


In [5]:
!apt-get install -y openjdk-11-jdk
!pip install git+https://github.com/salaniz/pycocoevalcap
!pip install nltk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-1k109bxk
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-1k109bxk
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done


In [6]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['JAVA_HOME'], 'bin')

In [11]:
import json

path = "/content/drive/MyDrive/image_captioning/results.json"

# Load your predictions list
with open(path, "r") as f:
    preds = json.load(f)

# Handle both list and dict formats safely
if isinstance(preds, dict) and "annotations" in preds:
    preds = preds["annotations"]

# Keep only one caption per image_id (e.g., the first occurrence)
unique_preds = {}
for p in preds:
    if p["image_id"] not in unique_preds:
        unique_preds[p["image_id"]] = p["caption"]

# Build clean JSON
clean_preds = {"annotations": [{"image_id": k, "caption": v} for k, v in unique_preds.items()]}

# Save to new file
clean_path = "/content/drive/MyDrive/image_captioning/results_clean.json"
with open(clean_path, "w") as f:
    json.dump(clean_preds, f)

print(f"✅ Cleaned predictions saved to {clean_path}")



✅ Cleaned predictions saved to /content/drive/MyDrive/image_captioning/results_clean.json


In [13]:
import json

path = "/content/drive/MyDrive/image_captioning/results.json"

# Load original predictions (list or dict)
with open(path, "r") as f:
    preds = json.load(f)

# Handle both formats
if isinstance(preds, dict) and "annotations" in preds:
    preds = preds["annotations"]

# Keep one caption per image_id
unique_preds = {}
for p in preds:
    if p["image_id"] not in unique_preds:
        unique_preds[p["image_id"]] = p["caption"]

# ✅ Save as a plain list
clean_preds = [{"image_id": k, "caption": v} for k, v in unique_preds.items()]

clean_path = "/content/drive/MyDrive/image_captioning/results_clean.json"
with open(clean_path, "w") as f:
    json.dump(clean_preds, f)

print(f"✅ Cleaned predictions saved to {clean_path} ({len(clean_preds)} captions)")


✅ Cleaned predictions saved to /content/drive/MyDrive/image_captioning/results_clean.json (143 captions)


In [15]:
import json

annFile = '/content/drive/MyDrive/image_captioning/datasets/coco2014/annotations/captions_val2014.json'
resFile = "/content/drive/MyDrive/image_captioning/results_clean.json"

# Load both
with open(annFile, "r") as f:
    refs = json.load(f)["annotations"]

with open(resFile, "r") as f:
    preds = json.load(f)

# Collect sets of IDs
ref_ids = {a["image_id"] for a in refs}
pred_ids = {p["image_id"] for p in preds}

# Check differences
print("Total refs:", len(ref_ids))
print("Total preds:", len(pred_ids))
print("Missing in preds:", len(ref_ids - pred_ids))
print("Extra in preds:", len(pred_ids - ref_ids))

# Align to intersection only
common_ids = ref_ids & pred_ids
print("Common IDs:", len(common_ids))

# Filter to only matching IDs
aligned_refs = [r for r in refs if r["image_id"] in common_ids]
aligned_preds = [p for p in preds if p["image_id"] in common_ids]

# Save aligned versions
import os
aligned_ref_path = "/content/drive/MyDrive/image_captioning/references_aligned.json"
aligned_pred_path = "/content/drive/MyDrive/image_captioning/results_aligned.json"

json.dump({"annotations": aligned_refs}, open(aligned_ref_path, "w"))
json.dump(aligned_preds, open(aligned_pred_path, "w"))

print("✅ Saved aligned files:")
print(aligned_ref_path)
print(aligned_pred_path)


Total refs: 40504
Total preds: 143
Missing in preds: 40361
Extra in preds: 0
Common IDs: 143
✅ Saved aligned files:
/content/drive/MyDrive/image_captioning/references_aligned.json
/content/drive/MyDrive/image_captioning/results_aligned.json


In [16]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

coco = COCO(aligned_ref_path)
cocoRes = coco.loadRes(aligned_pred_path)

cocoEval = COCOEvalCap(coco, cocoRes)
cocoEval.evaluate()

for metric, score in cocoEval.eval.items():
    print(f"{metric}: {score:.3f}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


KeyError: 'info'

In [17]:
import json

# Paths
aligned_ref_path = "/content/drive/MyDrive/image_captioning/references_aligned.json"
aligned_pred_path = "/content/drive/MyDrive/image_captioning/results_aligned.json"

# Load aligned refs
with open(aligned_ref_path, "r") as f:
    ref_data = json.load(f)

# ref_data currently has only {'annotations': [...]}
annotations = ref_data["annotations"]

# Build a minimal valid COCO structure
images = [{"id": ann["image_id"]} for ann in annotations]
images = [dict(t) for t in {tuple(d.items()) for d in images}]  # remove duplicates

ref_fixed = {
    "info": {"description": "Aligned references"},
    "licenses": [],
    "images": images,
    "annotations": annotations
}

fixed_ref_path = "/content/drive/MyDrive/image_captioning/references_final.json"
json.dump(ref_fixed, open(fixed_ref_path, "w"))

print(f"✅ Fixed references file saved to: {fixed_ref_path}")


✅ Fixed references file saved to: /content/drive/MyDrive/image_captioning/references_final.json


In [18]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

annFile = "/content/drive/MyDrive/image_captioning/references_final.json"
resFile = "/content/drive/MyDrive/image_captioning/results_aligned.json"

coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

cocoEval = COCOEvalCap(coco, cocoRes)
cocoEval.evaluate()

for metric, score in cocoEval.eval.items():
    print(f"{metric}: {score:.3f}")


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 1711, 'reflen': 1429, 'guess': [1711, 1568, 1425, 1282], 'correct': [683, 197, 53, 13]}
ratio: 1.1973407977598338
Bleu_1: 0.399
Bleu_2: 0.224
Bleu_3: 0.123
Bleu_4: 0.066
computing METEOR score...
METEOR: 0.142
computing Rouge score...
ROUGE_L: 0.324
computing CIDEr score...
CIDEr: 0.248
computing SPICE score...
SPICE: 0.081
Bleu_1: 0.399
Bleu_2: 0.224
Bleu_3: 0.123
Bleu_4: 0.066
METEOR: 0.142
ROUGE_L: 0.324
CIDEr: 0.248
SPICE: 0.081


In [21]:
import json

metrics = {
    "Bleu_1": 0.399,
    "Bleu_2": 0.224,
    "Bleu_3": 0.123,
    "Bleu_4": 0.066,
    "METEOR": 0.142,
    "ROUGE_L": 0.324,
    "CIDEr": 0.248,
    "SPICE": 0.081
}

with open("/content/drive/MyDrive/image_captioning/checkpoints/baseline_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("✅ Baseline metrics saved.")


✅ Baseline metrics saved.
